In [3]:
import sys
sys.path.append('C:/Anaconda3/Lib/site-packages')

import sqlite3
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

## European Football Kaggle Database
Author: Alex Galea
- +25,000 matches
- +10,000 players
- 11 European Countries with their lead championship
- Seasons 2008 to 2016
- Player's statistics sourced from EA Sports' FIFA video game series, including the weekly updates
- Team line up with squad formation (X, Y coordinates)
- Betting odds from up to 10 providers
- Detailed match events (goal types, possession, corner, cross, fouls, cards etc...) for +10,000 matches
---
I would like to develop an algorithm for identifying "good bets" where the better is likely to beat the odds. This is a supervised learning problem where the target is probably a binary classification (win/loss or over/under).

The idea here is not to try and predict the winner, but instead guess probabilities and identify situations where you can "beat the odds". For example: for a given team and season, compute the probabilities of winning the bet for each game and compare to the odds. Then look at the cases where there is the largest difference between predicted probabilites and odds.

In [ ]:
database = sqlite3.connect('database.sqlite')